In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from statsmodels.genmod.generalized_linear_model import SET_USE_BIC_LLF
from scipy.stats import nbinom
import random
import warnings
from statsmodels.tools.sm_exceptions import PerfectSeparationWarning

# Import custom modules and functions
from generate_hnb import generate_hnb
from generate_ZI import generate_ZI
from AIC_BIC import calculate_aic_bic

# Import model classes
from models.ZKIHurdlePoisson import ZKHurdlePoisson
from models.ZKIHurdleNB import ZKHurdleNB
from models.ZINB import ZINB_EM, predict_mean as ZINB_pred_mean
from models.ZIP import ZIP_EM, predict_mean as ZIP_pred_mean
from models.ZKINB import ZkINB_EM
from models.ZKIP import ZKIP_EM
from models.ZkICMP import ZkICMP

from sklearn.ensemble import RandomForestRegressor 
from sklearn.ensemble import HistGradientBoostingRegressor
from models.ZKHurdleRF import ZKHurdleRF

In [19]:
# warnings.filterwarnings('ignore', module='statsmodels')
warnings.filterwarnings('ignore', category=PerfectSeparationWarning)

class ModelEvaluator:
    """Class to evaluate and compare different count data models."""
    
    def __init__(self, X_train, X_test, y_train, y_test):
        self.X_train = X_train
        self.X_test = X_test
        self.y_train = y_train
        self.y_test = y_test
        self.n_train = X_train.shape[0]
        self.results = {}
    
    def evaluate_model(self, model_name, y_pred_test, y_pred_train, llf, k_params):
        """Calculate evaluation metrics for a model."""
        metrics = {
            'mse_test': mean_squared_error(self.y_test, y_pred_test),
            'mae_test': mean_absolute_error(self.y_test, y_pred_test),
            'r2_test': r2_score(self.y_test, y_pred_test),
            'mse_train': mean_squared_error(self.y_train, y_pred_train),
            'mae_train': mean_absolute_error(self.y_train, y_pred_train),
            'r2_train': r2_score(self.y_train, y_pred_train),
            'llf': llf,
            'aic': calculate_aic_bic(self.n_train, llf, k_params)[0],
            'bic': calculate_aic_bic(self.n_train, llf, k_params)[1]
        }
        self.results[model_name] = metrics
        return metrics
    
class ModelEvaluatorML:
    def __init__(self, X_train, X_test, y_train, y_test):
        self.X_train = X_train
        self.X_test = X_test
        self.y_train = y_train
        self.y_test = y_test
        self.n_train = X_train.shape[0]
        self.results = {}
    
    def evaluate_model(self, model_name, y_pred_test, y_pred_train):
        """Calculate evaluation metrics for a model."""
        metrics = {
            'mse_test': mean_squared_error(self.y_test, y_pred_test),
            'mae_test': mean_absolute_error(self.y_test, y_pred_test),
            'r2_test': r2_score(self.y_test, y_pred_test),
            'mse_train': mean_squared_error(self.y_train, y_pred_train),
            'mae_train': mean_absolute_error(self.y_train, y_pred_train),
            'r2_train': r2_score(self.y_train, y_pred_train)
        }
        self.results[model_name] = metrics
        return metrics

def define_model_parameters():
    """Define parameter counts for different models."""
    return {
        'poisson': 2,
        'negative_binomial': 3,  # if r known (2)
        'zk_hurdle_poisson': 4,  # 2+2
        'zk_hurdle_nb': 5, # 2+2+1 , 1 for alpha
        'zinb': 5,  # if r known (2+2+1)
        'zip': 4,  # 2+2
        'zkinb': 7,  # if r known (2+2+2+1)
        'zkip': 6,  # 2+2+2
        'zkicmp': 7  # 2+2+2+1
    }

def fit_models(X_train, X_test, y_train, y_test, k, alpha):
    # Configuration
    SET_USE_BIC_LLF(True)
    
    # Get parameter counts
    param_counts = define_model_parameters()
    
    # Initialize evaluator
    evaluator = ModelEvaluator(X_train, X_test, y_train, y_test)
    
    # 1. Poisson Model
    #print("Fitting Poisson model...")
    try:
        poisson_model = sm.GLM(y_train, X_train, family=sm.families.Poisson()).fit()
        poisson_pred = poisson_model.predict(X_test)
        poisson_pred_train = poisson_model.predict(X_train)
        evaluator.evaluate_model(
            'pois', poisson_pred, poisson_pred_train, poisson_model.llf, param_counts['poisson']
        )
    except Exception as e:
        print(e)
    
    # 2. Negative Binomial Model
    #print("Fitting Negative Binomial model...")
    try:
        nb_model = sm.NegativeBinomial(y_train, X_train).fit(disp=0)
        nb_pred = nb_model.predict(X_test)
        nb_pred_train = nb_model.predict(X_train)
        evaluator.evaluate_model(
            'nb', nb_pred, nb_pred_train, nb_model.llf, param_counts['negative_binomial']
        )
    except Exception as e:
        print(e)
    
    # 3. Zero-K Inflated Poisson Hurdle Model
    #print("Fitting Zero-K Inflated Poisson Hurdle model...")
    try:
        zkihurdle_model = ZKHurdlePoisson(k)
        zkihurdle_res = zkihurdle_model.fit(X_train, y_train)
        zkihurdle_ll = zkihurdle_model.loglikelihood(X_train, y_train)
        zkihurdle_pred = zkihurdle_model.predict_mean(X_test)
        zkihurdle_pred_train = zkihurdle_model.predict_mean(X_train)
        evaluator.evaluate_model(
            'zk_h_p', zkihurdle_pred, zkihurdle_pred_train, zkihurdle_ll, param_counts['zk_hurdle_poisson']
        )
    except Exception as e:
        print(e)

    # 5. ZKIHurdleNB Model
    try:
        zkihurdlenb_modle=ZKHurdleNB(k, alpha=alpha)
        zkihurdlenb_modle.fit(X_train, y_train)
        zkihurdlenb_pred=zkihurdlenb_modle.predict_mean(X_test)
        zkihurdlenb_pred_train=zkihurdlenb_modle.predict_mean(X_train)
        zkihurdlenb_ll=zkihurdlenb_modle.loglikelihood(X_train, y_train)
        evaluator.evaluate_model(
            'zk_h_nb', zkihurdlenb_pred, zkihurdlenb_pred_train, zkihurdlenb_ll, param_counts['zk_hurdle_nb']
        )
    except Exception as e:
        print(e)

    # 4. ZINB Model
    #print("Fitting ZINB model...")
    try:
        beta, gamma, zinb_ll = ZINB_EM(y_train.values, X_train.values, X_train.values, alpha=alpha)
        zinb_pred = ZINB_pred_mean(X_test.values, X_test.values, beta, gamma)
        zinb_pred_train = ZINB_pred_mean(X_train.values, X_train.values, beta, gamma)
        evaluator.evaluate_model(
            'zinb', zinb_pred, zinb_pred_train, zinb_ll, param_counts['zinb']
        )
    except Exception as e:
        print(e)
    
    # 5. ZIP Model
    #print("Fitting ZIP model...")
    try:
        beta, gamma, zip_ll = ZIP_EM(y_train.values, X_train.values, X_train.values)
        zip_pred = ZIP_pred_mean(X_test.values, X_test.values, beta, gamma)
        zip_pred_train = ZIP_pred_mean(X_train.values, X_train.values, beta, gamma)   
        evaluator.evaluate_model(
            'zip', zip_pred, zip_pred_train, zip_ll, param_counts['zip']
        )
    except Exception as e:
        print(e)

    # 6. ZKINB Model
    #print("Fitting ZKINB model...")
    try:
        zkinb_model = ZkINB_EM()
        zkinb_res = zkinb_model.fit_em(y_train.values, X_train.values, X_train.values, k)
        zkinb_pred = zkinb_model.predict(X_test.values, X_test.values)
        zkinb_pred_train = zkinb_model.predict(X_train.values, X_train.values)
        evaluator.evaluate_model(
            'zkinb', zkinb_pred, zkinb_pred_train, zkinb_res['final_loglik'], param_counts['zkinb']
        )
    except Exception as e:
        print(e)

    # 7. ZKIP Model
    #print("Fitting ZKIP model...")
    try:
        zkip_model = ZKIP_EM(k_inflated=k)
        zkip_res = zkip_model.fit(X_train.values, y_train.values)
        zkip_pred = zkip_model.predict_expected(X_test.values)
        zkip_pred_train = zkip_model.predict_expected(X_train.values)
        evaluator.evaluate_model(
            'zkip', zkip_pred, zkip_pred_train, zkip_res.final_loglik, param_counts['zkip']
        )
    except Exception as e:
        print(e)

    '''
    # 8. ZkICMP Model
    print("Fitting ZkICMP model...")
    zkicmp_model = ZkICMP(k=k)
    zkicmp_res = zkicmp_model.fit(X_train.values, y_train.values)
    pred_results = zkicmp_model.predict(X_test.values)
    _, _, zkicmp_pred, _ = pred_results
    evaluator.evaluate_model(
        'zkicmp', zkicmp_pred, -zkicmp_res.final_loglik, param_counts['zkicmp']
    )
    '''
    
    results = {}
    for model_name, metrics in evaluator.results.items():
        results[f'{model_name.upper()}_MSE_test'] = metrics['mse_test']
        results[f'{model_name.upper()}_MAE_test'] = metrics['mae_test']
        results[f'{model_name.upper()}_R2_test'] = metrics['r2_test']
        results[f'{model_name.upper()}_MSE_train'] = metrics['mse_train']
        results[f'{model_name.upper()}_MAE_train'] = metrics['mae_train']
        results[f'{model_name.upper()}_R2_train'] = metrics['r2_train']
        results[f'{model_name.upper()}_LLF'] = metrics['llf']
        results[f'{model_name.upper()}_AIC'] = metrics['aic']
        results[f'{model_name.upper()}_BIC'] = metrics['bic']
    
    return results

def fit_ML_models(X_train, X_test, y_train, y_test, k):
    # Initialize evaluator
    evaluator = ModelEvaluatorML(X_train, X_test, y_train, y_test)


    # RandomForestRegressor
    try:
        rf = RandomForestRegressor(
                n_estimators=300,
                criterion="poisson",
                min_samples_leaf=20,
                n_jobs=-1,
                random_state=42
            )
        rf.fit(X_train, y_train)
        y_pred_rf = rf.predict(X_test)
        y_pred_train_rf = rf.predict(X_train)
        evaluator. evaluate_model('RF', y_pred_rf, y_pred_train_rf)
    except Exception as e:
        print(e)
    
    # HistGradientBoostingRegressor with Poisson loss
    try:
        poisson_GB = HistGradientBoostingRegressor(
                    loss='poisson',        # This is the key - uses Poisson likelihood
                    random_state=42,
                    max_iter=100,
                    learning_rate=0.1,
                    max_depth=6,
                    min_samples_leaf=20    # Good for count data to prevent overfitting
                )
        poisson_GB.fit(X_train, y_train)
        y_pred_gb = poisson_GB.predict(X_test)
        y_pred_train_gb = poisson_GB.predict(X_train)
        evaluator. evaluate_model('GB', y_pred_gb, y_pred_train_gb)
    except Exception as e:
        print(e)

    # ZKHurdleRF
    #try:
    #    zk_h_rf = ZKHurdleRF(k=k)
    #    zk_h_rf.fit(X_train, y_train)
    #    y_pred_zkh_rf = zk_h_rf.predict_mean(X_test)
    #    y_pred_train_zkh_rf = zk_h_rf.predict_mean(X_train)
    #    evaluator.evaluate_model('zk_h_rf', y_pred_zkh_rf, y_pred_train_zkh_rf)
    #except Exception as e:
    #    print(e)

    results = {}
    for model_name, metrics in evaluator.results.items():
        results[f'{model_name.upper()}_MSE_test'] = metrics['mse_test']
        results[f'{model_name.upper()}_MAE_test'] = metrics['mae_test']
        results[f'{model_name.upper()}_R2_test'] = metrics['r2_test']
        results[f'{model_name.upper()}_MSE_train'] = metrics['mse_train']
        results[f'{model_name.upper()}_MAE_train'] = metrics['mae_train']
        results[f'{model_name.upper()}_R2_train'] = metrics['r2_train']
    
    return results

def check_for_nan(X_train, X_test, y_train, y_test):
    """Check for NaN values in data."""
    checks = {
        'X_train_nan': np.any(np.isnan(X_train.values)),
        'X_test_nan': np.any(np.isnan(X_test.values)),
        'y_train_nan': np.any(np.isnan(y_train.values)),
        'y_test_nan': np.any(np.isnan(y_test.values)),
        'X_train_inf': np.any(np.isinf(X_train.values)),
        'y_train_inf': np.any(np.isinf(y_train.values))
    }
    return checks

def main(n, rep, gen_m):

    k = 3
    test_size=0.3
    # Split data indexes
    train_ind, test_ind = train_test_split(
        np.arange(0, n), 
        test_size=test_size, random_state=42
    )

    results = []
    results_r2_minus = []
    problmatic_para = []
    err=[]
    beta0=gamma0=-1
    alpha0=1
    random.seed(42)
    for replication in range(0, rep):
        for beta in np.linspace(-2, 2, 5):  
            for gamma in np.linspace(-2, 2, 5):
                for alpha in np.linspace(-2, 2, 5):
                    # print(f"\nIteration: beta={beta:.2f}, gamma={gamma:.2f}, alpha={alpha:.2f}")
                    
                    """Generate and prepare the dataset for modeling."""
                    # Generate data - using the parameters from the loops   
                    if gen_m=='hnb':                 
                        df = generate_hnb(
                                n=n, k=k, beta0=beta0, beta1=beta, gamma0=gamma0, gamma1=gamma,
                                alpha0=alpha0, alpha1=alpha, r=10, cov_type="nbinary"
                            )
                    if gen_m=='zi':
                        df = generate_ZI(
                                n=n, k=k, beta0=beta0, beta1=beta, gamma0=gamma0, gamma1=gamma,
                                alpha0=alpha0, alpha1=alpha, r=10, cov_type="nbinary"
                            )
                        # Prepare features and target
                    X = df.loc[:, 'x'].values.reshape(-1, 1)
                    y_target = df.loc[:, 'y'].values
                        
                    # Add intercept
                    X = sm.add_constant(X)
                    X = pd.DataFrame(X, columns=['intercept', 'x'])
                    y_target = pd.Series(y_target, name='y')
                        
                    X_train = X.iloc[train_ind, :]
                    X_test = X.iloc[test_ind, :]
                    y_train = y_target.iloc[train_ind]
                    y_test = y_target.iloc[test_ind]

                    p_0 = (y_train == 0).mean()
                    p_k = (y_train == k).mean()
                    p_p = max(1 - p_0 - p_k, 0)
                    # Calculate statistics
                    y_mean = y_train.mean()
                    y_std = y_train.std()
                    n_unique = len(pd.Series(y_train).value_counts())
                    
                    checks = check_for_nan(X_train, X_test, y_train, y_test)
                    if any(checks.values()):
                        print(f"NaN/Inf found: {checks}")
                        print(f"y_mean: {y_mean}, y_std: {y_std}")
                        print(f"y_std^2 - y_mean: {y_std**2 - y_mean}")
                        continue  

                    if not min(p_0, p_k, p_p) >= 0.01:
                        problmatic_para.append({
                                'beta': beta,
                                'gamma': gamma,
                                'alpha': alpha,
                                'p_0': p_0,
                                'p_k': p_k,
                                'p_p': p_p,
                                'y_mean': y_mean,
                                'y_std': y_std,
                                'n_unique': n_unique
                            })
                                    
                    else:
                        # Estimate r for negative binomial
                        if y_std**2 > y_mean:
                            r_hat = y_mean**2 / max(y_std**2 - y_mean, 1e-9)
                            p0_nb = nbinom.pmf(0, r_hat, r_hat/(r_hat + y_mean))
                            pk_nb = nbinom.pmf(k, r_hat, r_hat/(r_hat + y_mean))
                            zero_inflated = p_0 > p0_nb 
                            k_inflated = p_k > pk_nb 
                            # Fit models
                            try:
                                model_results = fit_models(X_train, X_test, y_train, y_test, k, 1/r_hat)
                                ml_model_results=fit_ML_models(X_train, X_test, y_train, y_test, k)
                                results.append({
                                            'beta': beta,
                                            'gamma': gamma,
                                            'alpha': alpha,
                                            'p_0': p_0,
                                            'p_k': p_k,
                                            'p_p': p_p,
                                            'y_mean': y_mean,
                                            'y_std': y_std,
                                            'n_unique': n_unique,
                                            'r_hat': r_hat,
                                            'p0_nb': p0_nb,
                                            'pk_nb': pk_nb,
                                            'zero_inflated': zero_inflated,
                                            'k_inflated': k_inflated,
                                    } | model_results
                                      | ml_model_results)
                            except Exception as e:
                                print(e)
                                err.append({
                                    'beta': beta,
                                    'gamma': gamma,
                                    'alpha': alpha,
                                    'p_0': p_0,
                                    'p_k': p_k,
                                    'p_p': p_p,
                                    'y_mean': y_mean,
                                    'y_std': y_std,
                                    'n_unique': n_unique
                                })
                        else:
                            r_hat = np.nan
                            p0_nb = np.nan
                            pk_nb = np.nan
                            results_r2_minus.append({
                                            'beta': beta,
                                            'gamma': gamma,
                                            'alpha': alpha,
                                            'p_0': p_0,
                                            'p_k': p_k,
                                            'p_p': p_p,
                                            'y_mean': y_mean,
                                            'y_std': y_std,
                                            'n_unique': n_unique,
                                        })
        
    return pd.DataFrame(results), pd.DataFrame(results_r2_minus), pd.DataFrame(problmatic_para), pd.DataFrame(err)

#if __name__ == "__main__":
#    results_H, results_r2_minus_H, problmatic_para_H, err= main(300, 10)

In [20]:
n=500
rep=1
gen_m='zi' # hnb or zi
results, results_r2_minus, problmatic_para, err= main(n, rep, gen_m)
results.to_csv(f'results/results_{gen_m}_n={n}_rep={rep}.csv')
results_r2_minus.to_csv(f'results/results_r2_minus_{gen_m}_n={n}_rep={rep}.csv')
problmatic_para.to_csv(f'results/problmatic_para_{gen_m}_n={n}_rep={rep}.csv')
err.to_csv(f'results/err_{gen_m}_n={n}_rep={rep}.csv')

/Users/pasindumadusanka/.pyenv/versions/python_3_10_13-env/lib/python3.10/site-packages/statsmodels/base/model.py:595: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '
/Users/pasindumadusanka/Documents/GitHub/ResearchProject/ResearchProjectMain/Final Simulation/Simulation_Evalution/models/ZINB.py:201: RuntimeWarning: divide by zero encountered in log
  + r * np.log(prob_nz) + y_nz * np.log(1 - prob_nz)
/Users/pasindumadusanka/.pyenv/versions/python_3_10_13-env/lib/python3.10/site-packages/sklearn/metrics/_regression.py:501: RuntimeWarning: overflow encountered in square
  output_errors = np.average((y_true - y_pred) ** 2, axis=0, weights=sample_weight)
/Users/pasindumadusanka/.pyenv/versions/python_3_10_13-env/lib/python3.10/site-packages/sklearn/metrics/_regression.py:1196: RuntimeWarning: overflow encountered in square
  numerator = (weight * (y_true - y_pred) ** 2).sum(axis=0, dtype=np

Input contains infinity or a value too large for dtype('float64').
Input contains NaN.


/Users/pasindumadusanka/.pyenv/versions/python_3_10_13-env/lib/python3.10/site-packages/statsmodels/discrete/discrete_model.py:3379: RuntimeWarning: divide by zero encountered in log
  llf = coeff + size*np.log(prob) + endog*np.log(1-prob)
/Users/pasindumadusanka/.pyenv/versions/python_3_10_13-env/lib/python3.10/site-packages/statsmodels/discrete/discrete_model.py:3379: RuntimeWarning: invalid value encountered in multiply
  llf = coeff + size*np.log(prob) + endog*np.log(1-prob)
/Users/pasindumadusanka/Documents/GitHub/ResearchProject/ResearchProjectMain/Final Simulation/Simulation_Evalution/models/ZINB.py:201: RuntimeWarning: divide by zero encountered in log
  + r * np.log(prob_nz) + y_nz * np.log(1 - prob_nz)
/Users/pasindumadusanka/Documents/GitHub/ResearchProject/ResearchProjectMain/Final Simulation/Simulation_Evalution/models/ZIP.py:16: RuntimeWarning: divide by zero encountered in divide
  z = eta + (y - mu) / mu
/Users/pasindumadusanka/Documents/GitHub/ResearchProject/ResearchP

Input contains NaN.


/Users/pasindumadusanka/.pyenv/versions/python_3_10_13-env/lib/python3.10/site-packages/statsmodels/discrete/discrete_model.py:3379: RuntimeWarning: divide by zero encountered in log
  llf = coeff + size*np.log(prob) + endog*np.log(1-prob)
/Users/pasindumadusanka/.pyenv/versions/python_3_10_13-env/lib/python3.10/site-packages/statsmodels/discrete/discrete_model.py:3379: RuntimeWarning: invalid value encountered in multiply
  llf = coeff + size*np.log(prob) + endog*np.log(1-prob)
/Users/pasindumadusanka/.pyenv/versions/python_3_10_13-env/lib/python3.10/site-packages/statsmodels/discrete/discrete_model.py:3375: RuntimeWarning: divide by zero encountered in scalar divide
  size = 1/alpha * mu**Q
/Users/pasindumadusanka/.pyenv/versions/python_3_10_13-env/lib/python3.10/site-packages/statsmodels/discrete/discrete_model.py:3376: RuntimeWarning: invalid value encountered in divide
  prob = size/(size+mu)
/Users/pasindumadusanka/.pyenv/versions/python_3_10_13-env/lib/python3.10/site-packages/s

...

Input contains infinity or a value too large for dtype('float64').
Input contains NaN.


/Users/pasindumadusanka/.pyenv/versions/python_3_10_13-env/lib/python3.10/site-packages/statsmodels/discrete/discrete_model.py:3379: RuntimeWarning: divide by zero encountered in log
  llf = coeff + size*np.log(prob) + endog*np.log(1-prob)
/Users/pasindumadusanka/.pyenv/versions/python_3_10_13-env/lib/python3.10/site-packages/statsmodels/discrete/discrete_model.py:3379: RuntimeWarning: invalid value encountered in multiply
  llf = coeff + size*np.log(prob) + endog*np.log(1-prob)
/Users/pasindumadusanka/.pyenv/versions/python_3_10_13-env/lib/python3.10/site-packages/statsmodels/discrete/discrete_model.py:3375: RuntimeWarning: divide by zero encountered in scalar divide
  size = 1/alpha * mu**Q
/Users/pasindumadusanka/.pyenv/versions/python_3_10_13-env/lib/python3.10/site-packages/statsmodels/discrete/discrete_model.py:3376: RuntimeWarning: invalid value encountered in divide
  prob = size/(size+mu)
/Users/pasindumadusanka/.pyenv/versions/python_3_10_13-env/lib/python3.10/site-packages/s

Input contains infinity or a value too large for dtype('float64').


/Users/pasindumadusanka/Documents/GitHub/ResearchProject/ResearchProjectMain/Final Simulation/Simulation_Evalution/models/ZINB.py:201: RuntimeWarning: divide by zero encountered in log
  + r * np.log(prob_nz) + y_nz * np.log(1 - prob_nz)
/Users/pasindumadusanka/.pyenv/versions/python_3_10_13-env/lib/python3.10/site-packages/statsmodels/discrete/discrete_model.py:3379: RuntimeWarning: divide by zero encountered in log
  llf = coeff + size*np.log(prob) + endog*np.log(1-prob)
/Users/pasindumadusanka/.pyenv/versions/python_3_10_13-env/lib/python3.10/site-packages/statsmodels/discrete/discrete_model.py:3379: RuntimeWarning: invalid value encountered in multiply
  llf = coeff + size*np.log(prob) + endog*np.log(1-prob)
/Users/pasindumadusanka/.pyenv/versions/python_3_10_13-env/lib/python3.10/site-packages/statsmodels/discrete/discrete_model.py:3377: RuntimeWarning: invalid value encountered in subtract
  coeff = (gamma_ln(size+endog) - gamma_ln(endog+1) -
/Users/pasindumadusanka/.pyenv/versio

Input contains infinity or a value too large for dtype('float64').
Input contains NaN.


In [59]:
def main(n, rep, gen_m):
    k = 3
    test_size=0.3
    # Split data indexes
    train_ind, test_ind = train_test_split(
        np.arange(0, n), 
        test_size=test_size, random_state=42
    )

    results = []
    results_r2_minus = []
    problmatic_para = []
    err=[]
    beta0=np.log(0.1/0.9)
    beta=0
    for replication in range(0, rep):
            for gamma0 in np.linspace(-1, 1, 3):
                for alpha0 in np.linspace(-1, 1, 3):
                        for gamma in np.linspace(-1, 1, 3):
                            for alpha in np.linspace(-1, 1, 3):
                                # print(f"\nIteration: beta={beta:.2f}, gamma={gamma:.2f}, alpha={alpha:.2f}")
                                """Generate and prepare the dataset for modeling."""
                                # Generate data - using the parameters from the loops   
                                if gen_m=='hnb':                 
                                    df = generate_hnb(
                                            n=n, k=k, beta0=beta0, beta1=beta, gamma0=gamma0, gamma1=gamma,
                                            alpha0=alpha0, alpha1=alpha, r=10, cov_type="nbinary"
                                        )
                                if gen_m=='zi':
                                    df = generate_ZI(
                                            n=n, k=k, beta0=beta0, beta1=beta, gamma0=gamma0, gamma1=gamma,
                                            alpha0=alpha0, alpha1=alpha, r=10, cov_type="nbinary"
                                        )
                                    # Prepare features and target
                                X = df.loc[:, 'x'].values.reshape(-1, 1)
                                y_target = df.loc[:, 'y'].values

                                structural_zeros=(df['w']==0).mean()
                                structural_ks=(df['w']==k).mean()
                                from_nb=(df['w']==k+1).mean()
                                # Add intercept
                                X = sm.add_constant(X)
                                X = pd.DataFrame(X, columns=['intercept', 'x'])
                                y_target = pd.Series(y_target, name='y')
                                    
                                X_train = X.iloc[train_ind, :]
                                X_test = X.iloc[test_ind, :]
                                y_train = y_target.iloc[train_ind]
                                y_test = y_target.iloc[test_ind]

                                p_0 = (y_train == 0).mean()
                                p_k = (y_train == k).mean()
                                p_p = max(1 - p_0 - p_k, 0)
                                # Calculate statistics
                                y_mean = y_train.mean()
                                y_std = y_train.std()
                                n_unique = len(pd.Series(y_train).value_counts())
                                
                                checks = check_for_nan(X_train, X_test, y_train, y_test)
                                if any(checks.values()):
                                    print(f"NaN/Inf found: {checks}")
                                    print(f"y_mean: {y_mean}, y_std: {y_std}")
                                    print(f"y_std^2 - y_mean: {y_std**2 - y_mean}")
                                    continue  

                                if not min(p_0, p_k, p_p) >= 0.01:
                                    problmatic_para.append({
                                            'beta0': beta0,
                                            'gamma0': gamma0,
                                            'alpha0': alpha0,
                                            'beta': beta,
                                            'gamma': gamma,
                                            'alpha': alpha,
                                            'p_0': p_0,
                                            'p_k': p_k,
                                            'p_p': p_p,
                                            'structural_zeros': structural_zeros,
                                            'structural_ks': structural_ks,
                                            'from_nb': from_nb,
                                            'y_mean': y_mean,
                                            'y_std': y_std,
                                            'n_unique': n_unique
                                        })
                                                
                                else:
                                    # Estimate r for negative binomial
                                    if y_std**2 > y_mean:
                                        r_hat = y_mean**2 / max(y_std**2 - y_mean, 1e-9)
                                        p0_nb = nbinom.pmf(0, r_hat, r_hat/(r_hat + y_mean))
                                        pk_nb = nbinom.pmf(k, r_hat, r_hat/(r_hat + y_mean))
                                        zero_inflated = p_0 > p0_nb 
                                        k_inflated = p_k > pk_nb 
                                        # Fit models
                                        try:
                                            #model_results = fit_models(X_train, X_test, y_train, y_test, k, 1/r_hat)
                                            #ml_model_results=fit_ML_models(X_train, X_test, y_train, y_test, k)
                                            results.append({
                                                        'beta0': beta0,
                                                        'gamma0': gamma0,
                                                        'alpha0': alpha0,
                                                        'beta': beta,
                                                        'gamma': gamma,
                                                        'alpha': alpha,
                                                        'p_0': p_0,
                                                        'p_k': p_k,
                                                        'p_p': p_p,
                                                        'structural_zeros': structural_zeros,
                                                        'structural_ks': structural_ks,
                                                        'from_nb': from_nb,
                                                        'y_mean': y_mean,
                                                        'y_std': y_std,
                                                        'n_unique': n_unique,
                                                        'r_hat': r_hat,
                                                        'p0_nb': p0_nb,
                                                        'pk_nb': pk_nb,
                                                        'zero_inflated': zero_inflated,
                                                        'k_inflated': k_inflated,
                                                } )
                                                #| model_results
                                                #| ml_model_results)
                                        except Exception as e:
                                            print(e)
                                            err.append({
                                                'beta0': beta0,
                                                'gamma0': gamma0,
                                                'alpha0': alpha0,
                                                'beta': beta,
                                                'gamma': gamma,
                                                'alpha': alpha,
                                                'p_0': p_0,
                                                'p_k': p_k,
                                                'p_p': p_p,
                                                'structural_zeros': structural_zeros,
                                                'structural_ks': structural_ks,
                                                'from_nb': from_nb,
                                                'y_mean': y_mean,
                                                'y_std': y_std,
                                                'n_unique': n_unique
                                            })
                                    else:
                                        r_hat = np.nan
                                        p0_nb = np.nan
                                        pk_nb = np.nan
                                        results_r2_minus.append({
                                                        'beta0': beta0,
                                                        'gamma0': gamma0,
                                                        'alpha0': alpha0,
                                                        'beta': beta,
                                                        'gamma': gamma,
                                                        'alpha': alpha,
                                                        'p_0': p_0,
                                                        'p_k': p_k,
                                                        'p_p': p_p,
                                                        'structural_zeros': structural_zeros,
                                                        'structural_ks': structural_ks,
                                                        'from_nb': from_nb,
                                                        'y_mean': y_mean,
                                                        'y_std': y_std,
                                                        'n_unique': n_unique,
                                                    })
                    
    return pd.DataFrame(results), pd.DataFrame(results_r2_minus), pd.DataFrame(problmatic_para), pd.DataFrame(err)



In [60]:
n=500
rep=1
gen_m='zi' # hnb or zi
results, results_r2_minus, problmatic_para, err= main(n, rep, gen_m)
results.to_csv(f'results/results_{gen_m}_n={n}_rep={rep}.csv')
results_r2_minus.to_csv(f'results/results_r2_minus_{gen_m}_n={n}_rep={rep}.csv')
problmatic_para.to_csv(f'results/problmatic_para_{gen_m}_n={n}_rep={rep}.csv')
err.to_csv(f'results/err_{gen_m}_n={n}_rep={rep}.csv')

In [62]:
results

,beta0,gamma0,alpha0,beta,gamma,alpha,p_0,p_k,p_p,structural_zeros,structural_ks,from_nb,y_mean,y_std,n_unique,r_hat,p0_nb,pk_nb,zero_inflated,k_inflated
0,-2.197225,-1.0,-1.0,0,-1.0,-1.0,0.520000,0.342857,0.137143,0.102,0.318,0.580,1.231429,1.420535,6,1.928076,0.385867,0.084439,True,True
1,-2.197225,-1.0,-1.0,0,-1.0,0.0,0.505714,0.351429,0.142857,0.100,0.326,0.574,1.217143,1.374655,4,2.202769,0.379463,0.084533,True,True
2,-2.197225,-1.0,-1.0,0,-1.0,1.0,0.465714,0.325714,0.208571,0.120,0.310,0.570,1.305714,1.466228,8,2.019749,0.365270,0.090348,True,True
3,-2.197225,-1.0,-1.0,0,0.0,-1.0,0.494286,0.302857,0.202857,0.108,0.294,0.598,1.205714,1.384719,6,2.042550,0.387677,0.083016,True,True
4,-2.197225,-1.0,-1.0,0,0.0,0.0,0.517143,0.305714,0.177143,0.096,0.290,0.614,1.117143,1.324442,4,1.959185,0.413127,0.075686,True,True
5,-2.197225,-1.0,-1.0,0,0.0,1.0,0.502857,0.308571,0.188571,0.102,0.294,0.604,1.205714,1.376417,6,2.110523,0.385299,0.083284,True,True
6,-2.197225,-1.0,-1.0,0,1.0,-1.0,0.434286,0.337143,0.228571,0.098,0.332,0.570,1.448571,1.885379,9,0.996333,0.408858,0.084466,True,True
7,-2.197225,-1.0,-1.0,0,1.0,0.0,0.517143,0.325714,0.157143,0.096,0.300,0.604,1.165714,1.352477,4,2.048127,0.397418,0.079881,True,True
8,-2.197225,-1.0,-1.0,0,1.0,1.0,0.480000,0.405714,0.114286,0.088,0.358,0.554,1.368571,1.429784,5,2.771880,0.328803,0.098734,True,True
9,-2.197225,-1.0,0.0,0,-1.0,-1.0,0.374286,0.362857,0.262857,0.092,0.318,0.590,1.580000,1.520877,8,3.405424,0.273085,0.117490,True,True


In [ ]:
results['structural_zeros'].mean()

0.10170909090909092

In [ ]:
results.loc[(results['beta0']==0) & (results['beta']==0),:]

,beta0,gamma0,alpha0,beta,gamma,alpha,p_0,p_k,p_p,structural_zeros,structural_ks,from_nb,y_mean,y_std,n_unique,r_hat,p0_nb,pk_nb,zero_inflated,k_inflated
210,0.0,-1.0,-1.0,0.0,-1.0,-1.0,0.637143,0.317143,0.045714,0.492,0.300,0.208,1.008571,1.386565,4,1.112939,0.487735,0.063936,True,True
211,0.0,-1.0,-1.0,0.0,-1.0,0.0,0.634286,0.302857,0.062857,0.492,0.304,0.204,0.980000,1.363524,4,1.092360,0.496839,0.061893,True,True
212,0.0,-1.0,-1.0,0.0,-1.0,1.0,0.611429,0.294286,0.094286,0.536,0.266,0.198,1.085714,1.521035,8,0.960044,0.483688,0.067120,True,True
213,0.0,-1.0,-1.0,0.0,0.0,-1.0,0.657143,0.248571,0.094286,0.484,0.256,0.260,0.885714,1.312742,5,0.936618,0.536113,0.054646,True,True
214,0.0,-1.0,-1.0,0.0,0.0,0.0,0.665714,0.260000,0.074286,0.492,0.260,0.248,0.860000,1.302565,4,0.883974,0.548452,0.052646,True,True
215,0.0,-1.0,-1.0,0.0,0.0,1.0,0.685714,0.240000,0.074286,0.532,0.254,0.214,0.811429,1.277482,4,0.802426,0.570816,0.049012,True,True
216,0.0,-1.0,-1.0,0.0,1.0,-1.0,0.597143,0.294286,0.108571,0.486,0.268,0.246,1.080000,1.580468,6,0.822637,0.501694,0.064722,True,True
217,0.0,-1.0,-1.0,0.0,1.0,0.0,0.677143,0.271429,0.051429,0.510,0.278,0.212,0.868571,1.324179,4,0.852564,0.549405,0.053021,True,True
218,0.0,-1.0,-1.0,0.0,1.0,1.0,0.680000,0.285714,0.034286,0.508,0.278,0.214,0.891429,1.347686,3,0.859233,0.542524,0.054527,True,True
237,0.0,-1.0,0.0,0.0,-1.0,-1.0,0.602857,0.314286,0.082857,0.502,0.292,0.206,1.068571,1.412545,6,1.232145,0.463277,0.068768,True,True


In [38]:
results_r2_minus

,beta0,gamma0,alpha0,beta,gamma,alpha,p_0,p_k,p_p,structural_zeros,structural_ks,from_nb,y_mean,y_std,n_unique
0,-1.0,0.0,1.0,0.0,1.0,0.0,0.248571,0.562857,0.188571,0.256,0.500,0.244,2.220000,1.485421,9
1,-1.0,1.0,-1.0,0.0,1.0,-1.0,0.245714,0.708571,0.045714,0.248,0.664,0.088,2.231429,1.341699,7
2,-1.0,1.0,-1.0,0.0,1.0,0.0,0.325714,0.651429,0.022857,0.252,0.656,0.092,1.980000,1.406962,4
3,-1.0,1.0,0.0,0.0,-1.0,-1.0,0.297143,0.682857,0.020000,0.252,0.658,0.090,2.068571,1.375548,3
4,-1.0,1.0,0.0,0.0,1.0,0.0,0.314286,0.628571,0.057143,0.272,0.628,0.100,1.968571,1.386234,4
5,-1.0,1.0,0.0,1.0,0.0,0.0,0.308571,0.665714,0.025714,0.290,0.656,0.054,2.025714,1.388418,4
6,-1.0,1.0,0.0,1.0,0.0,1.0,0.325714,0.651429,0.022857,0.300,0.638,0.062,1.982857,1.405981,4
7,-1.0,1.0,1.0,-1.0,0.0,-1.0,0.294286,0.677143,0.028571,0.286,0.654,0.060,2.088571,1.379604,6
8,-1.0,1.0,1.0,-1.0,0.0,0.0,0.271429,0.671429,0.057143,0.298,0.636,0.066,2.180000,1.429990,8
9,-1.0,1.0,1.0,0.0,-1.0,-1.0,0.302857,0.651429,0.045714,0.276,0.628,0.096,2.040000,1.387046,5


In [39]:
problmatic_para

,beta0,gamma0,alpha0,beta,gamma,alpha,p_0,p_k,p_p,structural_zeros,structural_ks,from_nb,y_mean,y_std,n_unique
0,-1.0,1.0,-1.0,-1.0,0.0,-1.0,0.271429,0.720000,8.571429e-03,0.226,0.722,0.052,2.168571,1.338277,3
1,-1.0,1.0,-1.0,-1.0,0.0,0.0,0.374286,0.620000,5.714286e-03,0.312,0.638,0.050,1.865714,1.452878,3
2,-1.0,1.0,-1.0,-1.0,1.0,-1.0,0.294286,0.705714,0.000000e+00,0.282,0.718,0.000,2.117143,1.369120,2
3,-1.0,1.0,-1.0,-1.0,1.0,0.0,0.305714,0.694286,0.000000e+00,0.306,0.694,0.000,2.082857,1.384107,2
4,-1.0,1.0,-1.0,-1.0,1.0,1.0,0.340000,0.660000,0.000000e+00,0.332,0.668,0.000,1.980000,1.423161,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252,1.0,1.0,1.0,1.0,-1.0,1.0,0.700000,0.300000,5.551115e-17,0.720,0.280,0.000,0.900000,1.376741,2
253,1.0,1.0,1.0,1.0,0.0,-1.0,0.688571,0.311429,0.000000e+00,0.714,0.286,0.000,0.934286,1.391222,2
254,1.0,1.0,1.0,1.0,0.0,0.0,0.677143,0.322857,0.000000e+00,0.682,0.318,0.000,0.968571,1.404714,2
255,1.0,1.0,1.0,1.0,0.0,1.0,0.642857,0.357143,0.000000e+00,0.648,0.348,0.004,1.071429,1.439530,2


In [18]:
results[(results['p_p']>0.3)]

,beta0,gamma0,alpha0,beta,gamma,alpha,p_0,p_k,p_p,y_mean,y_std,n_unique,r_hat,p0_nb,pk_nb,zero_inflated,k_inflated
51,-1.0,-1.0,0.0,1.0,1.0,-1.0,0.385714,0.300000,0.314286,1.868571,2.046700,11,1.504717,0.296789,0.111053,True,True
55,-1.0,-1.0,1.0,-1.0,-1.0,0.0,0.348571,0.322857,0.328571,1.928571,1.782769,10,2.976241,0.226095,0.134903,True,True
56,-1.0,-1.0,1.0,-1.0,-1.0,1.0,0.305714,0.325714,0.368571,3.805714,5.362714,24,0.580430,0.309164,0.079674,False,True
58,-1.0,-1.0,1.0,-1.0,0.0,0.0,0.297143,0.377143,0.325714,2.162857,1.760533,9,4.994508,0.165789,0.159673,True,True
59,-1.0,-1.0,1.0,-1.0,0.0,1.0,0.337143,0.314286,0.348571,3.225714,5.032037,26,0.470917,0.378961,0.071828,False,True
65,-1.0,-1.0,1.0,0.0,-1.0,1.0,0.280000,0.320000,0.400000,4.077143,6.126863,29,0.496786,0.331924,0.072744,False,True
66,-1.0,-1.0,1.0,0.0,0.0,-1.0,0.385714,0.288571,0.325714,2.800000,5.708788,21,0.263173,0.524174,0.050200,False,True
67,-1.0,-1.0,1.0,0.0,0.0,0.0,0.337143,0.345714,0.317143,2.045714,1.800405,9,3.499872,0.199706,0.144723,True,True
68,-1.0,-1.0,1.0,0.0,0.0,1.0,0.348571,0.288571,0.362857,2.608571,3.188165,18,0.900583,0.293795,0.099860,True,True
69,-1.0,-1.0,1.0,0.0,1.0,-1.0,0.300000,0.345714,0.354286,3.417143,4.847561,25,0.581468,0.325903,0.080473,False,True


---

## **Data Generating Algorithm for ZkINB**

For $( i = 1  \to  n )$:

1. Generate covariate: $( x_i \sim N(0, 1) )$
2. Calculate zero-inflation probability:

   $
   \pi_{1i} = \frac{\exp(\beta_0 + \beta_1 x_i)}{1 + \exp(\beta_0 + \beta_1 x_i)}
   $
   
3. Calculate k-inflation probability:

   $
   \pi_{2i} = \frac{\exp(\gamma_0 + \gamma_1 x_i)}{1 + \exp(\gamma_0 + \gamma_1 x_i)}
   $
   
4. Calculate count mean parameter:

   $
   \lambda_i = \exp(\alpha_0 + \alpha_1 x_i)
   $
   
5. Generate latent uniform variable: $( u_i \sim \text{Uniform}(0, 1) )$

6. Generate response:
   $
   y_i = \begin{cases}
   0 & \text{if } u_i < \pi_{1i} \\
   k & \text{if } \pi_{1i} \leq u_i < \pi_{1i} + \pi_{2i} \\
   \text{NB}(r, \frac{r}{r + \lambda_i}) & \text{otherwise}
   \end{cases}
   $
   
   where $( \text{NB}(r, p) )$ = Negative Binomial with:
   - Dispersion parameter $( r )$
   - Success probability $( p = \frac{r}{r + \lambda_i} )$

---